In [1]:
print("hello world")
#hello can you see this new change
#please write your name just to make sure we can all edit this
#bea
#dilara
#gabi

hello world


In [3]:
#Imports
import numpy as np
import scipy as sci

In [3]:
"""
Class for quantum register for n number of qubits
"""
def tensor_Product(a,b):
    product= a+b
    return product

class Quantum_Register:
    def __init__(self,n):
        self.n = n 
        self.psi = np.zeros((2,)*n)

In [4]:
"""
Class for the quantum circuits 
"""
class Quantum_Circuit:
    def __init__(self):
        self = self

In [5]:
"""Testing block """
print("the tensor product is, ", tensor_Product(1,2))
newRegister= Quantum_Register(1)
print(newRegister.n)
print(newRegister.psi)

the tensor product is,  3
1
[0. 0.]


In [66]:
#define number of qubits, N

N = 3

In [67]:
def qubit_register(N):
    psi = (1/(2**(1/2)))*N*np.zeros((2**N, 1), dtype = complex)
    
    return psi
psi = qubit_register(N)
print(psi)

[[0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]]


In [72]:
def hadamard_gate(N):
    hadamard = (1/(np.sqrt(2)))*np.array((2**N)*[[1,1],[1,-1]])
    return hadamard
hadamard = hadamard_gate(N)
print(hadamard)

[[ 0.70710678  0.70710678]
 [ 0.70710678 -0.70710678]
 [ 0.70710678  0.70710678]
 [ 0.70710678 -0.70710678]
 [ 0.70710678  0.70710678]
 [ 0.70710678 -0.70710678]
 [ 0.70710678  0.70710678]
 [ 0.70710678 -0.70710678]
 [ 0.70710678  0.70710678]
 [ 0.70710678 -0.70710678]
 [ 0.70710678  0.70710678]
 [ 0.70710678 -0.70710678]
 [ 0.70710678  0.70710678]
 [ 0.70710678 -0.70710678]
 [ 0.70710678  0.70710678]
 [ 0.70710678 -0.70710678]]


In [1]:
def diffusion_operator(N):
    # Define superposition state
    s = np.ones((2**N, 1), dtype=complex) / np.sqrt(2**N)
    # Compute |s><s|
    ss_dagger = np.dot(s, s.T.conj())
    # ID Matric
    identity = np.eye(2**N)
    # Diffusion Operator
    diffusion = 2 * ss_dagger - identity
    return diffusion

In [5]:
fuse= diffusion_operator(3)
print(fuse)

[[-0.75+0.j  0.25+0.j  0.25+0.j  0.25+0.j  0.25+0.j  0.25+0.j  0.25+0.j
   0.25+0.j]
 [ 0.25+0.j -0.75+0.j  0.25+0.j  0.25+0.j  0.25+0.j  0.25+0.j  0.25+0.j
   0.25+0.j]
 [ 0.25+0.j  0.25+0.j -0.75+0.j  0.25+0.j  0.25+0.j  0.25+0.j  0.25+0.j
   0.25+0.j]
 [ 0.25+0.j  0.25+0.j  0.25+0.j -0.75+0.j  0.25+0.j  0.25+0.j  0.25+0.j
   0.25+0.j]
 [ 0.25+0.j  0.25+0.j  0.25+0.j  0.25+0.j -0.75+0.j  0.25+0.j  0.25+0.j
   0.25+0.j]
 [ 0.25+0.j  0.25+0.j  0.25+0.j  0.25+0.j  0.25+0.j -0.75+0.j  0.25+0.j
   0.25+0.j]
 [ 0.25+0.j  0.25+0.j  0.25+0.j  0.25+0.j  0.25+0.j  0.25+0.j -0.75+0.j
   0.25+0.j]
 [ 0.25+0.j  0.25+0.j  0.25+0.j  0.25+0.j  0.25+0.j  0.25+0.j  0.25+0.j
  -0.75+0.j]]
